In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
from netCDF4 import Dataset


plt.style.use(typhon.plots.styles('typhon'))

@FuncFormatter
def pres_formatter(x, pos):
    return '{:.1f}'.format(x / 1e2)


def profile_p(p, x, ax=None, **kwargs):
    """Plot atmospheric profile of arbitrary property.
    
    Parameters:
        p (ndarray): Pressure [Pa].
        x (ndarray): Atmospheric property.
        ax (AxesSubplot): Axes to plot in.
        **kwargs: Additional keyword arguments passed to `plt.plot`.
    """
    if ax is None:
        ax = plt.gca()
        
    # Determine min/max pressure of **all** data in plot.
    pmin = np.min((p.min(), *ax.get_ylim()))
    pmax = np.max((p.max(), *ax.get_ylim()))
    ax.set_ylim(pmax, pmin)  # implicitly invert yaxis
    
    # Only plot bottom and left spines.
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
        
    # Label and format for yaxis.
    ax.yaxis.set_major_formatter(pres_formatter)
    if ax.is_first_col():
        ax.set_ylabel('Pressure [hPa]')
    
    # Actual plot.
    return ax.plot(x, p, **kwargs)


def profile_z(z, x, ax=None, **kwargs):
    """Plot atmospheric profile of arbitrary property.
    
    Parameters:
        z (ndarray): Height [m].
        x (ndarray): Atmospheric property.
        ax (AxesSubplot): Axes to plot in.
        **kwargs: Additional keyword arguments passed to `plt.plot`.
    """
    if ax is None:
        ax = plt.gca()
        
    # Determine min/max pressure of **all** data in plot.
    zmin = np.min((z.min(), *ax.get_ylim()))
    zmax = np.max((z.max(), *ax.get_ylim()))
    ax.set_ylim(zmin, zmax)
    
    # Only plot bottom and left spines.
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
        
    # Label and format for yaxis.
    if ax.is_first_col():
        ax.set_ylabel('Height [km]')
    
    # Actual plot.
    return ax.plot(x, z / 1e3, **kwargs)

fascod_seasons = [
    'tropical',
    'midlatitude-summer',
    'midlatitude-winter',
    'subarctic-summer',
    'subarctic-winter',
]

# Simple timeseries

In [ ]:
for season in fascod_seasons:
    data = Dataset('results/{}.nc'.format(season))

    fig, ax = plt.subplots()
    sm = ax.pcolormesh(
        data.variables['time'][:] / 24,
        data.variables['plev'][:],
        data.variables['T'][:].T,
        cmap=plt.get_cmap('temperature', 20),
        rasterized=True,
        vmin=140,
        vmax=310,
    )
    fig.colorbar(sm, label='Temperature [K]')
    ax.invert_yaxis()
    ax.set_yscale('log')
    ax.set_ylim(1100e2, 10)
    ax.yaxis.set_major_formatter(pres_formatter)
    ax.set_title('FASCOD {}'.format(season))
    ax.set_xlabel('Days')
    ax.set_ylabel('Pressure [hPa]')
    fig.savefig('plots/{}.pdf'.format(season))

# Timeseries animation

In [ ]:
season = 'tropical'

vmin = 140
vmax = 310


for season in fascod_seasons:
    data = Dataset('results/{}.nc'.format(season))
    time = data.variables['time'][:] / 24
    p = data.variables['plev'][:]
    T = data.variables['T'][:].T
    
    for t in range(1, time.size, 1):
        fig, [ax2, ax1] = plt.subplots(1, 2, sharey=True, figsize=(16, 8), gridspec_kw={'width_ratios':[1, 3]})
        sm = ax1.pcolormesh(
            time[0:t+1],
            p,
            T[:, 0:t+1],
            cmap=plt.get_cmap('temperature', 10),
            rasterized=True,
            vmin=vmin,
            vmax=vmax,
        )
        fig.colorbar(sm, label='Temperature [K]', ax=ax1)
        sm = ax1.pcolormesh(
            time[t:],
            p,
            T[:, t:],
            cmap=plt.get_cmap('Greys_r', 10),
            rasterized=True,
            vmin=vmin,
            vmax=vmax,
        )
        ax1.axvline(t)
        ax1.invert_yaxis()
        ax1.yaxis.set_major_formatter(pres_formatter)
        fig.suptitle('FASCOD {}'.format(season))
        ax1.set_xlabel('Days')

        # temperature sounding
        profile_p(p, T[:, t], ax=ax2)
        ax2.set_yscale('log')
        ax2.set_ylim(1100e2, 10)
        ax2.yaxis.set_major_formatter(pres_formatter)
        ax2.set_xlim(vmin, vmax)
        ax2.set_xlabel('Temperature [K]')
        fig.savefig('plots/animation/{}_{:04d}.png'.format(season, t), dpi=300)
        plt.close(fig)